Imports



In [ ]:
! pip install pymorphy2

In [ ]:
import re
import numpy as np
import pandas as pd
from pymorphy2 import MorphAnalyzer
from  more_itertools import unique_everseen

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def clean(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('(https?://[^\s]+)', '', text)
    text = re.sub('@[^\s]+', '', text)
    text = re.sub('rt', '', text)
    text = re.sub('[^а-яА-Я]+', ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    return text

In [ ]:
def pymorphy_pos(text):
    pos = []
    text = word_tokenize(text)
    for word in text:
        if word != '.':
            ana = MorphAnalyzer().parse(word)
            if ana[0].tag.POS != None:
              pos.append(ana[0].tag.POS) 
    pos = ' '.join(pos)
    return pos

def pymorphy_lemmas(text):
    lemmas = []
    text = word_tokenize(text)
    for word in text:
        if word != '.':
            ana = MorphAnalyzer().parse(word)
            lemmas.append(ana[0].normal_form)        
    lemmas = ' '.join(lemmas)
    return lemmas

1 столбец

In [ ]:
data = pd.read_csv('dataset.csv')

In [90]:
data

,id,text,lemmas,pos
0,408906692693221377,да все таки он немного похож на него но мой ма...,да всё таки он немного похожий на оно но мыть ...,PRCL PRCL PRCL NPRO ADVB ADJS PREP NPRO CONJ V...
1,408906695083954177,ну ты идиотка я испугалась за тебя,ну ты идиотка я испугаться за ты,PRCL NPRO NOUN NPRO VERB PREP NPRO
2,408906695356973056,кто то в углу сидит и погибает от голода а мы ...,кто то в угол сидеть и погибать от голод а мы ...,NPRO CONJ PREP NOUN VERB CONJ VERB PREP NOUN C...
3,408906761416867842,вот что значит страшилка но блин посмотрев все...,вот что значит страшилка но блин посмотреть вс...,PRCL CONJ CONJ NOUN CONJ INTJ GRND PRCL NOUN P...
4,408906761769598976,ну любишь или нет я не знаю кто ты бля,ну любить или нет я не знать кто ты бля,PRCL VERB CONJ PRED NPRO PRCL VERB NPRO NPRO INTJ
...,...,...,...,...
9995,409766375364644864,ой я совсем забыла восстановить старый аккаунт...,ой я совсем забыть восстановить старый аккаунт...,INTJ NPRO ADVB VERB INFN ADJF NOUN CONJ NOUN A...
9996,409766474010886144,ну и где ночной твиттер все спят что ли так лю...,ну и где ночной твиттер всё спать что ли так л...,PRCL CONJ ADVB ADJF NOUN PRCL VERB CONJ PRCL C...
9997,409766520009793536,девочка моя спасибо тебе за этот вечер мне оче...,девочка мой спасибо ты за этот вечер я очень т...,NOUN ADJF INTJ NPRO PREP ADJF NOUN NPRO ADVB N...
9998,409766520425054208,долго объяснять все плохо вот,долго объяснять всё плохо вот,ADVB INFN PRCL PRED PRCL


In [ ]:
def pymorphy_lemmas(text):
    lemmas = []
    text = word_tokenize(text)
    for word in text:
        if word != '.':
            ana = MorphAnalyzer().parse(word)
            lemmas.append(ana[0].normal_form)        
    lemmas = ' '.join(lemmas)
    return lemmas

In [ ]:
# эта ищет поиск - нужно найти все предложения, где слово встречается в любой форме

def search_all(word_input):
  words = pymorphy_lemmas(word_input)
  words = words.split(' ')
  output = []
  for idx, sentence in enumerate(data['lemmas']):
    sen = word_tokenize(sentence)
    for i, word in enumerate(sen):
      if len(words) == 1 and word == words[0]:
        output.append(idx)
      elif len(words) == 2:
        try:
          if word == words[0] and sen[i+1] == words[1]:
            output.append(idx)
        except:
          pass
      elif len(words) == 3:
        try:
          if word == words[0] and sen[i+1] == words[1] and sen[i+2] == words[2]:
            output.append(idx)
        except:
          pass
  out = pd.DataFrame()
  out['id'] = data['id'].iloc[output]
  out['text'] = data['text'].iloc[output]
  return out

In [ ]:
# “поиска” - нужно найти предложения только с этой формой
# NOUN - найти все предложения с существительными

def search_exact(word_inputs):
  word_inputs = re.sub('"', '', word_inputs)
  words = word_inputs.split(' ')
  output = []
  for idx, sentence in enumerate(data['text']):
    sen = word_tokenize(sentence)
    for i, word in enumerate(sen):
      if len(words) == 1 and word == words[0]:
        output.append(idx)
      elif len(words) == 2:
        try:
          if word == words[0] and sen[i+1] == words[1]:
            output.append(idx)
        except:
          pass
      elif len(words) == 3:
        try:
          if word == words[0] and sen[i+1] == words[1] and sen[i+2] == words[2]:
            output.append(idx)
        except:
          pass
  out = pd.DataFrame()
  out['id'] = data['id'].iloc[output]
  out['text'] = data['text'].iloc[output]
  return out 

In [ ]:
# search(‘NOUN VERB ADVB’)

def search_pos(poses_all):
  poses = poses_all.upper()
  ids = []
  texts = []
  if len(poses.split(' '))>3:
    print('Эта программа не ищет такие длинные строчки.')
  else:
   for p in data['pos']:
      if poses in p:
        id = data.loc[data['pos']== p ].index.values
        ids.append(id[0])
   output = pd.DataFrame()
   output['id']=data['id'].iloc[ids]
   output['text']=data['text'].iloc[ids]
   return output

In [ ]:
def search_word_by_pos(word):                          
  splitted_word = word.split('+')
  word = pymorphy_lemmas(splitted_word[0])
  id_lines = []
  out = []
  fin_id_lines =[]
  for line in data['lemmas']:
    if word in line:
      id_line = data.loc[data['lemmas']== line ].index.values
      id_lines.append(id_line[0])
      splitted_line = line.split(' ')
      count_idx = 0
      for w in splitted_line:
        if w == word:
          count_idx = count_idx
        else:
          count_idx +=1
    for i in id_lines:
      pos_index = 0
      poses = data['pos'][i]
      for pos in poses:
        if pos == splitted_word[1]:
          pos_index = pos_index
        else:
          pos_index += 1
        if pos_index == count_idx:
          out.append(data['text'][i])
          fin_id_lines.append(id_line[0])
  unique_sentences = list(unique_everseen(out))
  unique_id_lines = list(unique_everseen(fin_id_lines))
  output = pd.DataFrame()
  output['id'] = data['id'].iloc[unique_id_lines]
  output['text']= unique_sentences
  return output

In [ ]:
def search(input):
  input = input.split()
  output = []
  type_of_input = [] #список, в котором лежат типы элементов инпута
  for i in input:
    if '"' in i:
      i = re.sub('"', '', i)
      output.append(search_exact(i).index.values)
      type_of_input.append('exactform') 
    elif '+' in i:
      output.append(search_word_by_pos(i).index.values)
      type_of_input.append('wordbypos')
    elif i.isupper():
      output.append(search_pos(i).index.values)
      type_of_input.append('pos')
    else:
      output.append(search_all(i).index.values)
      type_of_input.append('notexact')
  inter = set(output[0])
  for i in output[1:]:
    inter = inter.intersection(set(i))
  if len(input) == 1:
    inter = [i for i in inter]
    output = pd.DataFrame()
    output['id']=data['id'].iloc[inter]
    output['text']=data['text'].iloc[inter]
  else:
    idxs = []
    for index in inter:
      num = 0
      numbers = []
      for el in input: #берем элемент ввода
        el = re.sub('"', '', el)
        typeinput = type_of_input[num] #тип элемента ввода 
        num +=1
        if typeinput == 'exactform': #если точная форма, ищем номер элемента в нужной строчке столбца "текст"
          needtext = data['text'][index] #ищем нужное предложение
          text = needtext.split(' ') #разбиваем текст, чтобы вытащить номер нужного слова 
          number = text.index(el) #добавляем номер элемента ввода в список (чтобы потом проверить, подряд ли они)
          numbers.append(number)
        if typeinput == 'pos': 
          needtext = data['pos'][index] #ищем нужное предложение
          text = needtext.split(' ') #разбиваем текст, чтобы вытащить номер нужного слова 
          number = text.index(el)
          numbers.append(number)
        if typeinput == 'notexact': 
          element = pymorphy_lemmas(el)
          needtext = data['lemmas'][index] #ищем нужное предложение
          text = needtext.split(' ') #разбиваем текст, чтобы вытащить номер нужного слова 
          number = text.index(element)
          numbers.append(number)
        if typeinput == 'wordbypos': 
          el_sp = el.split('+')
          element_ = pymorphy_lemmas(el_sp[0])
          needtext = data['lemmas'][index] #ищем нужное предложение
          needpos= data['pos'][index]
          text = needtext.split(' ') #разбиваем текст, чтобы вытащить номер нужного слова 
          pos = needpos.split(' ')
          numtext = text.index(element_) 
          numpos = pos.index(el_sp[1])
          if numtext == numpos:
            numbers.append(numtext)
      n = 0
      for i in range(0,len(numbers)-1):
        if numbers[i] == (numbers[i+1])-1:
          n+=1
      if (n == len(numbers)-1):
        idxs.append(index)
    output = pd.DataFrame()
    output['id']=data['id'].iloc[idxs]
    output['text']=data['text'].iloc[idxs]

  return output

In [ ]:
search('люблю')

,id,text
3585,409049842615197696,тато я тебе люблю тв й дзв нок п дняв настр й
6147,409106804342530048,хочу чтоб меня любили но я не заслуживаю этого
4,408906761769598976,ну любишь или нет я не знаю кто ты бля
3078,409045014803931136,как ты тут просто такая милашка люблю тебяяя
5639,409060030659719168,ая думаю что она ко мне привязалась серьезно д...
...,...,...
5109,408916300837453824,меня никто не любит потому что я очкарик
1015,408917928156082176,черниковка очень любит путина первомайская
4602,409057602245754881,урааааа мой любимый вернулся с армии я самая с...
4603,409057602870714368,люблю зерненый творог и прочие полезные продук...


In [ ]:
search('"мимими"')

,id,text
6432,409206636365238272,ничего себе за что ты же такой мимими
5,408906762436481024,ох ну это конечно же чтобы у нее было много др...
8491,409513654284214273,прошел аж час я прочитал половину жуйколенты м...
2931,409043655992049664,вот так я не агрессивная я мимими
8372,409499991921922048,прошел аж час я прочитал половину жуйколенты м...
6618,409294520325787648,пока обновлял свой арч я со скоростью кб сек к...
4315,409055071981170688,сильна сильнааа это не уродско это необычно и ...
1372,408921568513765377,рома малая извращенка душик мимими все с тобой...
4541,409056996952776704,мимими спасибо но я так гонять не очень лююблю


In [ ]:
search('мой+NPRO')

,id,text
6,408906764114206720,у тебя есть ухажер нет мои уши не кто не жрет
7,408906764608749568,поприветствуем моего нового читателя
6157,409107243054157824,самое страшное оказалось правдой прости мои са...
2065,408928495582973952,оленьи и собачьи упряжки в лапландии моя новая...
2066,408928495889182721,ну по тупой причине я пообщалась с подругой ах...
...,...,...
4078,409053259815714817,вечер прошел замечательно рождественская истор...
8182,409435620764041217,она трогает моего мужа ета шлюха
8185,409435795129643008,засранцы передали авторизацию из за этого все ...
8187,409435882543140864,дота это такая херня из за которой хочется домой


In [ ]:
search('ADVB ADJF NOUN')

,id,text
124,408908174234628096,влюбляешься автоматически классные слова
128,408908177301053440,как и обещал действительно какие времена были
153,408908449209397248,у меня вообще такая тема что с любой стороны м...
203,408909047065493505,ты только первую часть сначала посмотри
257,408909598834192385,нафиг такую жизнь имя другое взяла и зачем тол...
...,...,...
7175,409342217892167680,это опять официальная информация а на самом деле
7208,409344790707179521,ох уж этот двойнойсмысл
8042,409425343184121856,где мои лет прожиты они безвозвратно
8174,409435120236769280,не очень удачная тренировка


In [ ]:
search('ADVB жить')

,id,text
6666,409298006937444352,как в этом мире тяжело жить
3480,409048859302977536,теперь живу в самом освещенном районе ущепните...
8740,409567580786278400,кажется приказал долго жить не отвечает ни ни
566,408913070518312960,устала так скучно жить хоть бы в школу пошла н...
5053,408911068569010176,так и медстраховки еще хорошей нет как они там...
6335,409198454150557696,бл вот везет а ты где живешь
5476,409049539254165504,транспорт от спбу ру приказал долго жить
5860,409086806273953793,ужасно жить в месте где тебе никогда не подаря...
8300,409492715890429952,как же невозможно жить в этом мире лжи ненавис...


In [ ]:
search('девочка+NOUN ADJF INTJ')

['девочка+NOUN', 'ADJF', 'INTJ']


,id,text
9997,409766520009793536,девочка моя спасибо тебе за этот вечер мне оче...
